In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
from sklearn.preprocessing    import OneHotEncoder
from sklearn.compose          import ColumnTransformer
from sklearn.preprocessing    import StandardScaler
from sklearn.pipeline         import Pipeline
from sklearn.metrics          import accuracy_score
from sklearn.base             import BaseEstimator, TransformerMixin
from sklearn.model_selection  import GridSearchCV
from sklearn.model_selection  import KFold

In [3]:
from sklearn.ensemble         import GradientBoostingClassifier
from sklearn.linear_model     import LogisticRegression
from sklearn.neighbors        import KNeighborsClassifier
from sklearn.ensemble         import RandomForestClassifier
from sklearn.neural_network   import MLPClassifier

In [4]:
#Read data
train_data = pd.read_csv("train.csv")
test_data  = pd.read_csv("test.csv")
# put aside coties
train_data_copy = train_data.copy()
test_data_copy  = test_data .copy()

In [5]:
# Look at data
train_data.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [6]:
# First glance at data: size, types
print(test_data.shape)
print(train_data.shape)
print(train_data.columns)
print(train_data.dtypes)

(418, 11)
(891, 12)
Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')
PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object


In [7]:
# First glance at data: cardinality
for item in train_data.columns: 
    print("\n{}\n{}".format(item, train_data[item].value_counts()))


PassengerId
891    1
293    1
304    1
303    1
302    1
      ..
591    1
590    1
589    1
588    1
1      1
Name: PassengerId, Length: 891, dtype: int64

Survived
0    549
1    342
Name: Survived, dtype: int64

Pclass
3    491
1    216
2    184
Name: Pclass, dtype: int64

Name
Pickard, Mr. Berk (Berk Trembisky)           1
Roebling, Mr. Washington Augustus II         1
Perreault, Miss. Anne                        1
Clifford, Mr. George Quincy                  1
Parkes, Mr. Francis "Frank"                  1
                                            ..
de Pelsmaeker, Mr. Alfons                    1
Dick, Mrs. Albert Adrian (Vera Gillespie)    1
Cameron, Miss. Clear Annie                   1
Attalah, Mr. Sleiman                         1
Todoroff, Mr. Lalio                          1
Name: Name, Length: 891, dtype: int64

Sex
male      577
female    314
Name: Sex, dtype: int64

Age
24.00    30
22.00    27
18.00    26
19.00    25
30.00    25
         ..
55.50     1
70.50     1
66.00

Form total dataset for some kinds of data analysis and engeenering

In [8]:
#First glance at data: 0/1 share
train_data.shape[0],train_data['Survived'].sum(),train_data['Survived'].sum()/train_data.shape[0]

(891, 342, 0.3838383838383838)

In [9]:
# drop depending column Survived
train_without_y = train_data.drop('Survived',axis =1)
print(train_without_y.columns,train_data.columns)
train_without_y.head(10)

Index(['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch',
       'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object') Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [10]:
# form common dataset
dataset_X = pd.concat([train_without_y,test_data])
dataset_X.reindex()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [11]:
# Check fo Nan: stat
Isnull_stat = pd.DataFrame({'total_N'      :dataset_X.isnull().sum(),
                            'total_Percent':dataset_X.isnull().mean(),
                            'train_N'      :train_without_y.isnull().sum(),
                            'train_Percent':train_without_y.isnull().mean()})
print('Nun stat:')
print('(number of passagers:',dataset_X.shape[0],')')
Isnull_stat

Nun stat:
(number of passagers: 1309 )


,total_N,total_Percent,train_N,train_Percent
PassengerId,0,0.000000,0,0.000000
Pclass,0,0.000000,0,0.000000
Name,0,0.000000,0,0.000000
Sex,0,0.000000,0,0.000000
Age,263,0.200917,177,0.198653
SibSp,0,0.000000,0,0.000000
Parch,0,0.000000,0,0.000000
Ticket,0,0.000000,0,0.000000
Fare,1,0.000764,0,0.000000
Cabin,1014,0.774637,687,0.771044


Missing data check

In [12]:
# List of columns with NaN
columns_nun = [ item for item in dataset_X.columns if (dataset_X[item].isnull().values.any() == True)]
columns_nun

['Age', 'Fare', 'Cabin', 'Embarked']

In [13]:
# Select columns with many Nan
lines_N = dataset_X.shape[0]
print(lines_N)
treshold_nun = 0.05
columns_many_nun = [ item for item in columns_nun if (dataset_X[item].isnull().values.sum()/lines_N > treshold_nun)]
columns_many_nun

1309


['Age', 'Cabin']

In [14]:
# Select columns with a little Nan
columns_few_nun  = [item for item in columns_nun if (item not in columns_many_nun)]
columns_few_nun

['Fare', 'Embarked']

In [15]:
# columns_nun dtypes
dataset_X[columns_nun].dtypes

Age         float64
Fare        float64
Cabin        object
Embarked     object
dtype: object

In [16]:
# First fill "few" gaps. Numerical var
mean_tr = train_data['Fare'].mean()
#train_data.loc[:,'Fare'] = train_data['Fare'].fillna(mean_tr)
#test_data.loc [:,'Fare'] = test_data ['Fare'].fillna(mean_tr)
train_data['Fare'].fillna(mean_tr, inplace = True)
test_data ['Fare'].fillna(mean_tr, inplace = True)
print(mean_tr)
print(train_data['Fare'].isnull().sum())
print(test_data ['Fare'].isnull().sum())

32.2042079685746
0
0


In [17]:
# First fill "few" gaps. Categorial var
more_often_embarked_s = train_data['Embarked'].mode()
more_often_embarked   = more_often_embarked_s[0] 
train_data['Embarked'].fillna(more_often_embarked, inplace = True)
test_data ['Embarked'].fillna(more_often_embarked, inplace = True)

print('more_oftem_embarked =  ',more_often_embarked)
print(train_data['Embarked'].isnull().sum())
print(test_data ['Embarked'].isnull().sum())
train_data['Embarked'].value_counts(),train_data['Embarked'].isnull().sum()

more_oftem_embarked =   S
0
0


(S    646
 C    168
 Q     77
 Name: Embarked, dtype: int64,
 0)

In [18]:
# Start with "many gap" var. Age
median_age = train_data['Age'].median()
print('median_age: ',median_age)
train_data['Age'].fillna(median_age, inplace = True)
test_data ['Age'].fillna(median_age, inplace = True)
train_data['Age'].isnull().sum(),test_data['Age'].isnull().sum()

median_age:  28.0


(0, 0)

Many Nan: Cabin

In [19]:
# Cabin: stat
value_count = dataset_X['Cabin'].value_counts()
print(len(value_count))
print(value_count)
print('N of Lines:',dataset_X.shape[0],'  N of Nun:',dataset_X['Cabin'].isnull().sum())

186
C23 C25 C27        6
B57 B59 B63 B66    5
G6                 5
F33                4
C22 C26            4
                  ..
B73                1
B3                 1
B36                1
B82 B84            1
C97                1
Name: Cabin, Length: 186, dtype: int64
N of Lines: 1309   N of Nun: 1014


In [20]:
# 'Cabin' is mixed variable - need to be divided
train_data['Cabin_letter'] = train_data['Cabin'].str[0]
test_data ['Cabin_letter'] = test_data ['Cabin'].str[0]
print(train_data['Cabin_letter'].unique())
print(train_data['Cabin_letter'].value_counts())
print(train_data['Cabin_letter'].isnull().sum(),
      train_data.shape[0],
      train_data.shape[0]-train_data['Cabin_letter'].isnull().sum())

[nan 'C' 'E' 'G' 'D' 'A' 'B' 'F' 'T']
C    59
B    47
D    33
E    32
A    15
F    13
G     4
T     1
Name: Cabin_letter, dtype: int64
687 891 204


In [21]:
train_data['Cabin_n'] = train_data['Cabin'].str.extract('(\d+)')
test_data ['Cabin_n'] = test_data ['Cabin'].str.extract('(\d+)')
print(train_data['Cabin_n'].unique(),len(train_data['Cabin_n'].unique()))
print(train_data['Cabin_n'].value_counts())
print(train_data['Cabin_n'].isnull().sum(),
      train_data.shape[0],
      train_data.shape[0]-train_data['Cabin_n'].isnull().sum())

[nan '85' '123' '46' '6' '103' '56' '23' '78' '33' '30' '52' '28' '83'
 '73' '31' '5' '10' '26' '110' '58' '101' '69' '47' '86' '2' '19' '7' '49'
 '4' '32' '80' '36' '15' '93' '35' '87' '77' '67' '94' '125' '99' '118'
 '22' '106' '65' '54' '57' '34' '18' '124' '91' '40' '128' '37' '50' '82'
 '96' '44' '104' '111' '92' '38' '21' '12' '63' '14' '20' '79' '25' '95'
 '39' '70' '16' '68' '41' '9' '48' '126' '71' '51' '62' '24' '90' '45' '8'
 '121' '11' '3' '17' '102' '42' '148'] 93
33     7
6      6
2      5
23     5
22     5
      ..
103    1
15     1
128    1
110    1
62     1
Name: Cabin_n, Length: 92, dtype: int64
691 891 200


In [22]:
# change 'Embarked' and 'Sex' type (for correct corr work):
#train_data['Embarked']  = train_data['Embarked'].astype('string')
#test_data ['Embarked']  = test_data ['Embarked'].astype('string')

#train_data['Sex']       = train_data['Sex'].astype('string')
#test_data ['Sex']       = test_data ['Sex'].astype('string')
# different version of python let to different result including "TypeError: data type "string" not understood"

In [23]:
# to fill gaps in 'Cabin_letter' and 'Cabin_n' lets look at correlation. Hypophesis is that 'CabinLetter' is in correlation with class, fare and ore may be ticket number  
train_data_drop_na_for_corr = train_data.dropna().copy()
#print(train_data_drop_na_for_corr.isnull().sum(),
#      train_data_drop_na_for_corr.columns,
#      train_data_drop_na_for_corr.dtypes)
train_data_drop_na_for_corr['Cabin_letter'] = train_data_drop_na_for_corr['Cabin_letter'].map({
    'A':0,
    'B':1,
    'C':2,
    'D':3,
    'E':4,
    'F':5,
    'G':6,
    'T':7})
train_data_drop_na_for_corr['Sex_n'] = train_data_drop_na_for_corr['Sex'].map({
    'male'  :0,
    'female':1,
    })
train_data_drop_na_for_corr['Cabin_n'] = train_data_drop_na_for_corr['Cabin_n'].astype('int')
train_data_drop_na_for_corr['Sex_n'] = train_data_drop_na_for_corr['Sex_n'].astype('int')
#print(train_data_drop_na_for_corr.dtypes)
correlation_s = train_data_drop_na_for_corr.corr(method = 'spearman')
#correlation_k = train_data_drop_na_for_corr.corr(method = 'kendall')
correlation_p = train_data_drop_na_for_corr.corr(method = 'pearson')
print(f"\n\n'Spearman:'\n{correlation_s.columns}\n{correlation_s[['Cabin_letter','Cabin_n']]}")
#print(f"\n\n'Kendall :'\n{correlation_k.columns}\n{correlation_k[['Cabin_letter','Cabin_n']]}")
print(f"\n\n'Person  :'\n{correlation_p.columns}\n{correlation_p[['Cabin_letter','Cabin_n']]}")



'Spearman:'
Index(['PassengerId', 'Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare',
       'Cabin_letter', 'Cabin_n', 'Sex_n'],
      dtype='object')
              Cabin_letter   Cabin_n
PassengerId      -0.069055  0.040026
Survived          0.047256 -0.069445
Pclass            0.553154 -0.079801
Age              -0.086763  0.047201
SibSp             0.078738 -0.045723
Parch             0.049733 -0.127942
Fare             -0.350580  0.104461
Cabin_letter      1.000000 -0.099209
Cabin_n          -0.099209  1.000000
Sex_n             0.077802 -0.052294


'Person  :'
Index(['PassengerId', 'Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare',
       'Cabin_letter', 'Cabin_n', 'Sex_n'],
      dtype='object')
              Cabin_letter   Cabin_n
PassengerId      -0.081550  0.035941
Survived          0.042262 -0.063747
Pclass            0.621100 -0.036776
Age              -0.174590  0.034167
SibSp             0.052532 -0.064264
Parch             0.037476 -0.117619
Fare             -0.

In [24]:
train_data_drop_na_for_corr.groupby(['Pclass'])['Cabin_letter'].mean()
# Plass correlates with Cabin_letter 

Pclass
1    2.011429
2    4.538462
3    5.083333
Name: Cabin_letter, dtype: float64

In [25]:
# Joint distribution 'Pclass','Cabin_letter'
Pclass_Сabin_letter = train_data_drop_na_for_corr[['Pclass','Cabin_letter']].copy()
Pclass_Сabin_letter['Count'] = 1
#print(Pclass_Сabin_letter)
Class_Cabin_l_stat = pd.pivot_table(Pclass_Сabin_letter,
              index   = 'Pclass',
              columns = 'Cabin_letter',
              values  = 'Count',                     
              aggfunc = np.sum)
Class_Cabin_l_stat.fillna(0,inplace = True)
Class_Cabin_l_stat,Class_Cabin_l_stat.sum(),Class_Cabin_l_stat.sum(axis = 1),Class_Cabin_l_stat.sum().sum()

(Cabin_letter     0     1     2     3     4    5    6
 Pclass                                              
 1             15.0  47.0  59.0  29.0  25.0  0.0  0.0
 2              0.0   0.0   0.0   1.0   4.0  8.0  0.0
 3              0.0   0.0   0.0   0.0   3.0  5.0  4.0,
 Cabin_letter
 0    15.0
 1    47.0
 2    59.0
 3    30.0
 4    32.0
 5    13.0
 6     4.0
 dtype: float64,
 Pclass
 1    175.0
 2     13.0
 3     12.0
 dtype: float64,
 200.0)

In [26]:
print(f"Pclass - survived stat\n{train_data.groupby(['Pclass'])['Survived'].mean()}")
cabin_letter_stat          = train_data['Cabin_letter'].value_counts(dropna =False).sort_values(ascending = False)
survived_cabin_letter_stat = train_data.groupby(['Cabin_letter'], dropna =False)['Survived'].mean()
to_print = pd.DataFrame({
    'N'              :cabin_letter_stat ,
    'SurvivedPercent':survived_cabin_letter_stat
    })
to_print

Pclass - survived stat
Pclass
1    0.629630
2    0.472826
3    0.242363
Name: Survived, dtype: float64


,N,SurvivedPercent
NaN,687,0.299854
C,59,0.593220
B,47,0.744681
D,33,0.757576
E,32,0.750000
A,15,0.466667
F,13,0.615385
G,4,0.500000
T,1,0.000000
NaN,687,0.299854


In [27]:
# Conclusion: Potentially may be it is reasonable to fill "Cabin_letter" individually for classes 1-3
# But: unknown Cabin is correlated with resulting var  - 'Survived'. 
# So being in task condition, it is better to encode Nan in Cabin as separate class 'Unknown'

In [28]:
# Replace Nan for 'Unknown'
train_data.loc[:,'Cabin_letter'] = train_data.loc[:,'Cabin_letter'].fillna('Unknown')
test_data.loc [:,'Cabin_letter'] = test_data.loc[:,'Cabin_letter'].fillna('Unknown')

#train_data['Cabin_letter'] = train_data['Cabin_letter'].astype('string')
#test_data ['Cabin_letter'] = test_data ['Cabin_letter'].astype('string')


In [29]:
# Replace Nan for '0'
train_data['Cabin_n'] = train_data['Cabin_n'].fillna(0)
test_data ['Cabin_n'] = test_data['Cabin_n'].fillna(0)
train_data['Cabin_n'] = train_data['Cabin_n'].astype(np.float64).astype('int64')
test_data ['Cabin_n'] = test_data ['Cabin_n'].astype(np.float64).astype('int64')

In [30]:
# Drop 'Cabin'
train_data.drop('Cabin',axis =1,inplace= True)
test_data. drop('Cabin',axis =1,inplace= True)

In [31]:
# Check for Nan affer replacement
train_data.isnull().sum()

PassengerId     0
Survived        0
Pclass          0
Name            0
Sex             0
Age             0
SibSp           0
Parch           0
Ticket          0
Fare            0
Embarked        0
Cabin_letter    0
Cabin_n         0
dtype: int64

In [32]:
# NaN replacement is finished 

In [33]:
# Ticket is mixed variable - need to be worked out

value_count = dataset_X['Ticket'].value_counts()
print(len(value_count))
print(value_count)

929
CA. 2343              11
CA 2144                8
1601                   8
PC 17608               7
347082                 7
                      ..
C.A. 15185             1
SOTON/O.Q. 3101314     1
AQ/3. 30631            1
350408                 1
3101297                1
Name: Ticket, Length: 929, dtype: int64


In [34]:
# Extract numarical part and fill Nan for mean
train_data['Ticket_n'] = train_data['Ticket'].str.extract('(\d+)')
test_data ['Ticket_n'] = test_data ['Ticket'].str.extract('(\d+)')
#print(train_data['Ticket_n'].dtypes)
#for el in train_data['Ticket_n']: print(el)
train_data['Ticket_n'] = train_data['Ticket_n'].astype(np.float64)
#train_data['Ticket_n'] = train_data['Ticket_n'].astype('int32')
test_data ['Ticket_n'] = test_data ['Ticket_n'].astype(np.float64)
#.astype('int32')
mean_ticker_n =train_data.Ticket_n.mean()
mean_ticker_n = int(mean_ticker_n)
print('train isnull before:',train_data['Ticket_n'].isnull().sum())
print('test  isnull before:',test_data ['Ticket_n'].isnull().sum())
print(mean_ticker_n, type(mean_ticker_n))
train_data['Ticket_n'].fillna(mean_ticker_n,inplace = True)
test_data ['Ticket_n'].fillna(mean_ticker_n,inplace = True)
train_data['Ticket_n'] = train_data['Ticket_n'].astype('int64')
test_data ['Ticket_n'] = test_data ['Ticket_n'].astype('int64')
print('isnull check:',train_data['Ticket_n'].isnull().sum())
print('isnull check:',train_data['Ticket_n'].isnull().sum())

train isnull before: 4
test  isnull before: 0
227846 <class 'int'>
isnull check: 0
isnull check: 0


In [35]:
#Check Letters in Ticket 
#train_data['Ticket_str'] = train_data['Ticket'].str.extract('([a-z+,A-z+])')
#test_data ['Ticket_str'] = test_data ['Ticket'].str.extract('([a-z+,A-z+])')
#print(train_data['Ticket_str'].value_counts(dropna = False))
#train_data['Ticket_str'].nunique(),test_data['Ticket_str'].nunique()
#temp_df = train_data['Ticket'].str.extract('([a-z+,A-z+])')

In [36]:
# Decision: don't use letters from Tickets

In [37]:
# Drop 'Name','Ticket' from data
print(train_data.columns)
train_data.drop(['Name','Ticket'], axis=1, inplace = True)
test_data .drop(['Name','Ticket'], axis=1, inplace = True)
print(train_data.columns)

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Embarked', 'Cabin_letter', 'Cabin_n',
       'Ticket_n'],
      dtype='object')
Index(['PassengerId', 'Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch',
       'Fare', 'Embarked', 'Cabin_letter', 'Cabin_n', 'Ticket_n'],
      dtype='object')


In [38]:
# Check for Nan, look data after steps above 
train_data.columns, train_data.isnull().sum(), train_data.head(10)

(Index(['PassengerId', 'Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch',
        'Fare', 'Embarked', 'Cabin_letter', 'Cabin_n', 'Ticket_n'],
       dtype='object'),
 PassengerId     0
 Survived        0
 Pclass          0
 Sex             0
 Age             0
 SibSp           0
 Parch           0
 Fare            0
 Embarked        0
 Cabin_letter    0
 Cabin_n         0
 Ticket_n        0
 dtype: int64,
    PassengerId  Survived  Pclass     Sex   Age  SibSp  Parch     Fare  \
 0            1         0       3    male  22.0      1      0   7.2500   
 1            2         1       1  female  38.0      1      0  71.2833   
 2            3         1       3  female  26.0      0      0   7.9250   
 3            4         1       1  female  35.0      1      0  53.1000   
 4            5         0       3    male  35.0      0      0   8.0500   
 5            6         0       3    male  28.0      0      0   8.4583   
 6            7         0       1    male  54.0      0      0  51.8625

Check for duplicates

In [39]:
# Check duplicated lines
data_temp = pd.concat([train_data.drop('Survived',axis = 1),test_data])
data_temp[data_temp.duplicated(keep = False) == True].shape
# no duplicated lines

(0, 11)

In [40]:
#Check duplicated columns
data_temp_trans = data_temp.transpose()
data_temp_trans[data_temp_trans.duplicated(keep = False) == True].shape, data_temp_trans.index 
# no duplicated columns

((0, 1309),
 Index(['PassengerId', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare',
        'Embarked', 'Cabin_letter', 'Cabin_n', 'Ticket_n'],
       dtype='object'))

Correlation between features

In [41]:
# Correlation between features after first steps
correlation_p = data_temp.corr( method = 'pearson')
correlation_s = data_temp.corr( method = 'spearman')
print(f"\n\n'Spearman:'\n{correlation_s.columns}\n{correlation_s}")
print(f"\n\n'Pearson  :'\n{correlation_p.columns}\n{correlation_p}")
#abs(Correlation) between features are less them 0,7 (for  Spearman)
# Correlation between Pclass and Cabin_n, Fear and Plcass.



'Spearman:'
Index(['PassengerId', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Cabin_n',
       'Ticket_n'],
      dtype='object')
             PassengerId    Pclass       Age     SibSp     Parch      Fare  \
PassengerId     1.000000 -0.039921  0.016007 -0.032295 -0.006191 -0.002831   
Pclass         -0.039921  1.000000 -0.345030 -0.066679 -0.028752 -0.708022   
Age             0.016007 -0.345030  1.000000 -0.102364 -0.182357  0.174245   
SibSp          -0.032295 -0.066679 -0.102364  1.000000  0.438373  0.444830   
Parch          -0.006191 -0.028752 -0.182357  0.438373  1.000000  0.399712   
Fare           -0.002831 -0.708022  0.174245  0.444830  0.399712  1.000000   
Cabin_n         0.001892 -0.666682  0.261585  0.104277  0.079090  0.542392   
Ticket_n       -0.028697  0.237422 -0.067573 -0.097199 -0.055146 -0.246642   

              Cabin_n  Ticket_n  
PassengerId  0.001892 -0.028697  
Pclass      -0.666682  0.237422  
Age          0.261585 -0.067573  
SibSp        0.104277 -0.0971

In [42]:
# Rare elements in 'Cabin_letter': stat
data_temp = pd.concat([train_data.drop(['Survived'], axis=1),test_data])
# list of feature to check rare values
n_rows = data_temp.shape[0]
list_to_check_rare = ['Cabin_letter']
for item in list_to_check_rare:
    serial_temp = data_temp[item].value_counts().sort_values(ascending = False)
    percent     = pd.Series(serial_temp/n_rows)
    data_frame_temp = pd.DataFrame({'N':serial_temp,'Percent':percent})
    print("{}:\n{}".format(item,data_frame_temp))    

Cabin_letter:
            N   Percent
Unknown  1014  0.774637
C          94  0.071811
B          65  0.049656
D          46  0.035141
E          41  0.031322
A          22  0.016807
F          21  0.016043
G           5  0.003820
T           1  0.000764


In [43]:
# Rare elements in 'Cabin_letter': replace
column = 'Cabin_letter'
treshold_rare_rate = 0.02
data_total = pd.concat([train_data.drop(['Survived'], axis=1),test_data])
len_total = data_temp.shape[0]
treshold_rare_rate_N = treshold_rare_rate*len_total
ind_list = list()
count = data_total[column].value_counts()
#print(count.index)
ind = count[count<treshold_rare_rate_N].index
ind_list=ind.to_list()
print('label to encode:',ind_list)

for item in ind_list:
    train_data.loc[train_data[column] == item,column] = 'rare'
    test_data. loc[test_data[column]  == item,column] = 'rare'

print(f'{train_data[column].value_counts()}\n\n{test_data[column].value_counts()}')


label to encode: ['A', 'F', 'G', 'T']
Unknown    687
C           59
B           47
D           33
rare        33
E           32
Name: Cabin_letter, dtype: int64

Unknown    327
C           35
B           18
rare        16
D           13
E            9
Name: Cabin_letter, dtype: int64


In [44]:
train_data.columns,train_data.dtypes

(Index(['PassengerId', 'Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch',
        'Fare', 'Embarked', 'Cabin_letter', 'Cabin_n', 'Ticket_n'],
       dtype='object'),
 PassengerId       int64
 Survived          int64
 Pclass            int64
 Sex              object
 Age             float64
 SibSp             int64
 Parch             int64
 Fare            float64
 Embarked         object
 Cabin_letter     object
 Cabin_n           int64
 Ticket_n          int64
 dtype: object)

In [45]:
# Look at correlation X with y

corr_X_y = train_data.corr(method = 'spearman')
print('Spearman corr\n{}'.format(corr_X_y))
# no strong correlations (except Pclass and Fare)

Spearman corr
             PassengerId  Survived    Pclass       Age     SibSp     Parch  \
PassengerId     1.000000 -0.005007 -0.034091  0.034584 -0.061161  0.001235   
Survived       -0.005007  1.000000 -0.339668 -0.037004  0.088879  0.138266   
Pclass         -0.034091 -0.339668  1.000000 -0.317406 -0.043019 -0.022801   
Age             0.034584 -0.037004 -0.317406  1.000000 -0.145316 -0.217488   
SibSp          -0.061161  0.088879 -0.043019 -0.145316  1.000000  0.450014   
Parch           0.001235  0.138266 -0.022801 -0.217488  0.450014  1.000000   
Fare           -0.013975  0.323736 -0.688032  0.126006  0.447113  0.410074   
Cabin_n         0.026237  0.308564 -0.674438  0.225499  0.056378  0.076375   
Ticket_n       -0.020400 -0.139273  0.233018 -0.063132 -0.065214 -0.032394   

                 Fare   Cabin_n  Ticket_n  
PassengerId -0.013975  0.026237 -0.020400  
Survived     0.323736  0.308564 -0.139273  
Pclass      -0.688032 -0.674438  0.233018  
Age          0.126006  0.2254

Encoding

In [46]:
# form lists for different encodeing
cat_text_to_encode_list      = [column for column in train_data.columns if train_data[column].dtypes == 'object']
num_to_standard_list         = ['Age','SibSp','Parch','Fare','Cabin_n','Ticket_n']

#check
list_left = [column for column in train_data.columns if (column not in cat_text_to_encode_list)&(column not in num_to_standard_list)]
list_left

['PassengerId', 'Survived', 'Pclass']

In [47]:
train_data_cat = train_data[cat_text_to_encode_list]
ohe = OneHotEncoder(drop = 'if_binary')
sc  = StandardScaler()
transformers = ColumnTransformer([
    ('OneHotEncoder' ,ohe,cat_text_to_encode_list),
    ('StandardScaler',sc, num_to_standard_list)], remainder = 'passthrough')


In [48]:
# Prepare X_train, etc.
PassengerIdTrain = train_data['PassengerId']
PassengerIdTest  = test_data ['PassengerId']
test_data_copy = test_data.copy()
y_train = train_data['Survived']
X_train = train_data.drop(['PassengerId','Survived'], axis = 1)
X_test  = test_data.drop('PassengerId',axis = 1)

In [49]:
gbm     = GradientBoostingClassifier(max_depth = 5, n_estimators = 30, random_state = 0)
log_reg = LogisticRegression()
knn     = KNeighborsClassifier()
forest  = RandomForestClassifier()
n_net   = MLPClassifier()

In [50]:
params = [{'clf':[gbm],    'clf__max_depth':[1,2,3,5,7],'clf__n_estimators':[10,30,50,100]}, 
          {'clf':[log_reg],'clf__C': [0.1,1,10]},
          {'clf':[knn],    'clf__n_neighbors':[3,4,5], 'clf__p':[1,2]},
          {'clf':[forest], 'clf__max_depth':[3,5,7]}
         ]


In [51]:
pipe = Pipeline(steps = [('transformers',transformers),('clf',gbm)])

In [52]:
search = GridSearchCV(pipe,params,
                      #scoring = accuracy_score,
                      #cv = 4
                     )
search.fit(X_train,y_train)
best   = search.best_estimator_
best

Pipeline(steps=[('transformers',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('OneHotEncoder',
                                                  OneHotEncoder(drop='if_binary'),
                                                  ['Sex', 'Embarked',
                                                   'Cabin_letter']),
                                                 ('StandardScaler',
                                                  StandardScaler(),
                                                  ['Age', 'SibSp', 'Parch',
                                                   'Fare', 'Cabin_n',
                                                   'Ticket_n'])])),
                ('clf', GradientBoostingClassifier(random_state=0))])

In [53]:
search.best_params_

{'clf': GradientBoostingClassifier(random_state=0),
 'clf__max_depth': 3,
 'clf__n_estimators': 100}

In [54]:
y_pred = best.predict(X_test)

In [55]:
y_train.mean(),y_pred.mean()

(0.3838383838383838, 0.361244019138756)

In [56]:
PassegerId_y_pred = pd.DataFrame({'PassengerId':PassengerIdTest,'Survived':y_pred})
PassegerId_y_pred.index = PassegerId_y_pred['PassengerId']
print(PassegerId_y_pred)


             PassengerId  Survived
PassengerId                       
892                  892         0
893                  893         0
894                  894         0
895                  895         0
896                  896         1
...                  ...       ...
1305                1305         0
1306                1306         1
1307                1307         0
1308                1308         0
1309                1309         0

[418 rows x 2 columns]


In [57]:
PassegerId_y_pred.to_csv("submission_grid_search.csv", index = False)

In [59]:
search.best_score_

0.8361684765551441